# Predict Future Sales - Modeling

Tutorial Link -> https://www.kaggle.com/code/werooring/ch9-modeling

In [1]:
!pip install lightgbm==3.2.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.6 MB/s eta 0:00:00
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 4.2.0
    Uninstalling lightgbm-4.2.0:
      Successfully uninstalled lightgbm-4.2.0


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/competitive-data-science-predict-future-sales/items.csv
/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv
/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv
/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv
/kaggle/input/competitive-data-science-predict-future-sales/shops.csv
/kaggle/input/competitive-data-science-predict-future-sales/test.csv


In [3]:
import warnings

warnings.filterwarnings(action='ignore') # ignore warnings

data_path = '/kaggle/input/competitive-data-science-predict-future-sales/'

items = pd.read_csv(data_path + 'items.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')
item_categories = pd.read_csv(data_path + 'item_categories.csv')
sales_train = pd.read_csv(data_path + 'sales_train.csv')
shops = pd.read_csv(data_path + 'shops.csv')
test = pd.read_csv(data_path + 'test.csv')

## Feature Engineering 1: Convert to Korean + Data Downcasting

In [4]:
sales_train = sales_train.rename(columns={'date': '날짜', 
                                          'date_block_num': '월ID',
                                          'shop_id': '상점ID',
                                          'item_id': '상품ID',
                                          'item_price': '판매가',
                                          'item_cnt_day': '판매량'})

shops = shops.rename(columns={'shop_name': '상점명',
                              'shop_id': '상점ID'})

items = items.rename(columns={'item_name': '상품명',
                              'item_id': '상품ID',
                              'item_category_id': '상품분류ID'})

item_categories = item_categories.rename(columns=
                                         {'item_category_name': '상품분류명',
                                          'item_category_id': '상품분류ID'})

test = test.rename(columns={'shop_id': '상점ID',
                            'item_id': '상품ID'})

In [5]:
def downcast(df, verbose=True):
    start_mem = df.memory_usage().sum() / 1024**2 # calculate start mem usage, change Byte -> Mega Byte by 1024**2
    for col in df.columns:
        dtype_name = df[col].dtype.name
        if dtype_name == 'object':
            pass
        elif dtype_name == 'bool':
            df[col] = df[col].astype('int8')
        elif dtype_name.startswith('int') or (df[col].round() == df[col]).all():
            df[col] = pd.to_numeric(df[col], downcast='integer')
        else:
            df[col] = pd.to_numeric(df[col], downcast='float')
    end_mem = df.memory_usage().sum() / 1024**2 # calculate end mem usage
    if verbose: # if verbose = True -> print detailed info
        print('{:.1f}% compressed'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

## Feature Engineering 2: Feature Engineering Individual Data

**sales_train: remove outlier, pre-process**

In [6]:
# extract '판매가' > 0 data
sales_train = sales_train[sales_train['판매가'] > 0]
# extract '판매가' < 50000 data
sales_train = sales_train[sales_train['판매가'] < 50000]

# extract '판매량' > 0 data
sales_train = sales_train[sales_train['판매량'] > 0]
# extract '판매가' < 1000 data
sales_train = sales_train[sales_train['판매량'] < 1000]

In [7]:
print(shops['상점명'][0], '||', shops['상점명'][57])
print(shops['상점명'][1], '||', shops['상점명'][58])
print(shops['상점명'][10], '||', shops['상점명'][11])
print(shops['상점명'][39], '||', shops['상점명'][40])

!Якутск Орджоникидзе, 56 фран || Якутск Орджоникидзе, 56
!Якутск ТЦ "Центральный" фран || Якутск ТЦ "Центральный"
Жуковский ул. Чкалова 39м? || Жуковский ул. Чкалова 39м²
РостовНаДону ТРК "Мегацентр Горизонт" || РостовНаДону ТРК "Мегацентр Горизонт" Островной


In [8]:
# sales_train change '상점ID'
sales_train.loc[sales_train['상점ID'] == 0, '상점ID'] = 57
sales_train.loc[sales_train['상점ID'] == 1, '상점ID'] = 58
sales_train.loc[sales_train['상점ID'] == 10, '상점ID'] = 11
sales_train.loc[sales_train['상점ID'] == 39, '상점ID'] = 40

# test change '상점ID'
test.loc[test['상점ID'] == 0, '상점ID'] = 57
test.loc[test['상점ID'] == 1, '상점ID'] = 58
test.loc[test['상점ID'] == 10, '상점ID'] = 11
test.loc[test['상점ID'] == 39, '상점ID'] = 40

**shops: create sub features, encoding**

In [9]:
shops['도시'] = shops['상점명'].apply(lambda x: x.split()[0])

In [10]:
shops['도시'].unique()

array(['!Якутск', 'Адыгея', 'Балашиха', 'Волжский', 'Вологда', 'Воронеж',
       'Выездная', 'Жуковский', 'Интернет-магазин', 'Казань', 'Калуга',
       'Коломна', 'Красноярск', 'Курск', 'Москва', 'Мытищи', 'Н.Новгород',
       'Новосибирск', 'Омск', 'РостовНаДону', 'СПб', 'Самара', 'Сергиев',
       'Сургут', 'Томск', 'Тюмень', 'Уфа', 'Химки', 'Цифровой', 'Чехов',
       'Якутск', 'Ярославль'], dtype=object)

In [11]:
shops.loc[shops['도시'] =='!Якутск', '도시'] = 'Якутск'

In [12]:
from sklearn.preprocessing import LabelEncoder

# create label encoder
label_encoder = LabelEncoder()
# encoding '도시' feature label
shops['도시'] = label_encoder.fit_transform(shops['도시'])

In [13]:
# remove '상점명' feature
shops = shops.drop('상점명', axis=1)

shops.head()

,상점ID,도시
0,0,29
1,1,29
2,2,0
3,3,1
4,4,2


**items: create sub features**

In [14]:
items = items.drop(['상품명'], axis=1)

In [15]:
# 상품이 맨 처음 팔린 날을 피처로 추가
items['첫 판매월'] = sales_train.groupby('상품ID').agg({'월ID': 'min'})['월ID']

items.head()

,상품ID,상품분류ID,첫 판매월
0,0,40,20.0
1,1,76,15.0
2,2,40,19.0
3,3,40,18.0
4,4,40,20.0


In [16]:
items[items['첫 판매월'].isna()]

,상품ID,상품분류ID,첫 판매월
83,83,40,NaN
140,140,45,NaN
168,168,44,NaN
173,173,45,NaN
204,204,44,NaN
...,...,...,...
21974,21974,61,NaN
21975,21975,61,NaN
22022,22022,40,NaN
22035,22035,40,NaN


In [17]:
items['첫 판매월'] = items['첫 판매월'].fillna(34)

**item_categories: create sub features, encoding**

In [18]:
item_categories['대분류'] = item_categories['상품분류명'].apply(lambda x: x.split()[0])  

In [19]:
item_categories['대분류'].value_counts()

대분류
Игры          14
Книги         13
Подарки       12
Игровые        8
Аксессуары     7
Музыка         6
Программы      6
Карты          5
Кино           5
Служебные      2
Чистые         2
PC             1
Билеты         1
Доставка       1
Элементы       1
Name: count, dtype: int64

In [20]:
def make_etc(x):
    if len(item_categories[item_categories['대분류']==x]) >= 5:
        return x
    else:
        return 'etc'

# if '대분류' unique val < 5 -> 'etc'
item_categories['대분류'] = item_categories['대분류'].apply(make_etc)

In [21]:
item_categories.head()

,상품분류명,상품분류ID,대분류
0,PC - Гарнитуры/Наушники,0,etc
1,Аксессуары - PS2,1,Аксессуары
2,Аксессуары - PS3,2,Аксессуары
3,Аксессуары - PS4,3,Аксессуары
4,Аксессуары - PSP,4,Аксессуары


In [22]:
# create label encoder 
label_encoder = LabelEncoder()

# '대분류' feature label encoding
item_categories['대분류'] = label_encoder.fit_transform(item_categories['대분류'])

# remove '상품분류명' feature
item_categories = item_categories.drop('상품분류명', axis=1)

## Feature Engineering 3: Data Combination, Create Sub Features

**Data Combination**

In [23]:
from itertools import product

train = []
# 월ID, 상점ID, 상품ID combination
for i in sales_train['월ID'].unique():
    all_shop = sales_train.loc[sales_train['월ID']==i, '상점ID'].unique()
    all_item = sales_train.loc[sales_train['월ID']==i, '상품ID'].unique()
    train.append(np.array(list(product([i], all_shop, all_item))))

idx_features = ['월ID', '상점ID', '상품ID'] 
train = pd.DataFrame(np.vstack(train), columns=idx_features)

**create sub features**

In [24]:
group = sales_train.groupby(idx_features).agg({'판매량': 'sum',
                                               '판매가': 'mean'})
group = group.reset_index()
group = group.rename(columns={'판매량': '월간 판매량', '판매가': '평균 판매가'})

train = train.merge(group, on=idx_features, how='left')

train.head()

,월ID,상점ID,상품ID,월간 판매량,평균 판매가
0,0,59,22154,1.0,999.0
1,0,59,2552,NaN,NaN
2,0,59,2554,NaN,NaN
3,0,59,2555,NaN,NaN
4,0,59,2564,NaN,NaN


In [25]:
import gc

# group var -> garbate. collection
del group
gc.collect();

In [26]:
# add feature: product sales
group = sales_train.groupby(idx_features).agg({'판매량': 'count'})
group = group.reset_index()
group = group.rename(columns={'판매량': '판매건수'})

train = train.merge(group, on=idx_features, how='left')

# garbage collection
del group, sales_train
gc.collect()

train.head()

,월ID,상점ID,상품ID,월간 판매량,평균 판매가,판매건수
0,0,59,22154,1.0,999.0,1.0
1,0,59,2552,NaN,NaN,NaN
2,0,59,2554,NaN,NaN,NaN
3,0,59,2555,NaN,NaN,NaN
4,0,59,2564,NaN,NaN,NaN


## Feature Engineering 5: Combine Data

**Add test data**

In [27]:
test['월ID'] = 34

# concat test and train set
all_data = pd.concat([train, test.drop('ID', axis=1)],
                     ignore_index=True,
                     keys=idx_features)
# missing value -> 0
all_data = all_data.fillna(0)

all_data.head()

,월ID,상점ID,상품ID,월간 판매량,평균 판매가,판매건수
0,0,59,22154,1.0,999.0,1.0
1,0,59,2552,0.0,0.0,0.0
2,0,59,2554,0.0,0.0,0.0
3,0,59,2555,0.0,0.0,0.0
4,0,59,2564,0.0,0.0,0.0


**merge all data**

In [28]:
# merge all data
all_data = all_data.merge(shops, on='상점ID', how='left')
all_data = all_data.merge(items, on='상품ID', how='left')
all_data = all_data.merge(item_categories, on='상품분류ID', how='left')

# data downcasting
all_data = downcast(all_data)

76.2% compressed


In [29]:
# garbage collection
del shops, items, item_categories
gc.collect();

## Feature Engineering 6: Create Lag Feature

**create monthly mean sales derived feature by idx features**

In [30]:
def add_mean_features(df, mean_features, idx_features):
    # check idx feature
    assert (idx_features[0] == '월ID') and \
           len(idx_features) in [2, 3]
    
    # set derived feature
    if len(idx_features) == 2:
        feature_name = idx_features[1] + '별 평균 판매량'
    else:
        feature_name = idx_features[1] + ' ' + idx_features[2] + '별 평균 판매량'
    
    # group by index feature -> get monthly mean sales
    group = df.groupby(idx_features).agg({'월간 판매량': 'mean'})
    group = group.reset_index()
    group = group.rename(columns={'월간 판매량': feature_name})
    
    # merge df & group 
    df = df.merge(group, on=idx_features, how='left')
    # data downcasting
    df = downcast(df, verbose=False)
    # add new feature_name to mean_features list
    mean_features.append(feature_name)
    
    # garbage collection 
    del group
    gc.collect()
    
    return df, mean_features

In [31]:
# list for idx features include derived features with `상품ID` 
item_mean_features = []

# group by ['월ID', '상품ID'] -> create monthly mean sales derived feature
all_data, item_mean_features = add_mean_features(df=all_data,
                                                 mean_features=item_mean_features,
                                                 idx_features=['월ID', '상품ID'])

# group by ['월ID', '상품ID', '도시'] -> create monthly mean sales derived feature
all_data, item_mean_features = add_mean_features(df=all_data,
                                                 mean_features=item_mean_features,
                                                 idx_features=['월ID', '상품ID', '도시'])

In [32]:
item_mean_features

['상품ID별 평균 판매량', '상품ID 도시별 평균 판매량']

In [33]:
# list for idx features include derived features with `상점ID` 
shop_mean_features = []

# group by ['월ID', '상점ID', '상품분류ID'] -> create monthly mean sales derived feature
all_data, shop_mean_features = add_mean_features(df=all_data, 
                                                 mean_features=shop_mean_features,
                                                 idx_features=['월ID', '상점ID', '상품분류ID'])

In [34]:
shop_mean_features

['상점ID 상품분류ID별 평균 판매량']

**how to create lag features**

In [35]:
def add_lag_features(df, lag_features_to_clip, idx_features, 
                     lag_feature, nlags=3, clip=False):
    # copy needed df sections for lag feature creation
    df_temp = df[idx_features + [lag_feature]].copy() 

    # create lag feature
    for i in range(1, nlags+1):
        # lag feature name
        lag_feature_name = lag_feature +'_시차' + str(i)
        # set df_temp's column name
        df_temp.columns = idx_features + [lag_feature_name]
        # add 1 to df_temp's date_block_num feature
        df_temp['월ID'] += 1
        # merge df & df_temp by idx_feature
        df = df.merge(df_temp.drop_duplicates(), 
                      on=idx_features, 
                      how='left')
        # missing value -> 0
        df[lag_feature_name] = df[lag_feature_name].fillna(0)
        if clip: 
            lag_features_to_clip.append(lag_feature_name)
    
    # data down casting
    df = downcast(df, False)
    # garbage collection
    del df_temp
    gc.collect()
    
    return df, lag_features_to_clip

**lag feature creation 1: 월간 판매량**

In [36]:
lag_features_to_clip = [] 
idx_features = ['월ID', '상점ID', '상품ID'] 

# create 3 month lag feature of monthly sales by idx_features
all_data, lag_features_to_clip = add_lag_features(df=all_data, 
                                                  lag_features_to_clip=lag_features_to_clip,
                                                  idx_features=idx_features,
                                                  lag_feature='월간 판매량', 
                                                  nlags=3,
                                                  clip=True) # limit value to 1-20

In [37]:
all_data.head().T

,0,1,2,3,4
월ID,0.000000,0.000000,0.000000,0.000000,0.000000
상점ID,59.000000,59.000000,59.000000,59.000000,59.000000
상품ID,22154.000000,2552.000000,2554.000000,2555.000000,2564.000000
월간 판매량,1.000000,0.000000,0.000000,0.000000,0.000000
평균 판매가,999.000000,0.000000,0.000000,0.000000,0.000000
판매건수,1.000000,0.000000,0.000000,0.000000,0.000000
도시,30.000000,30.000000,30.000000,30.000000,30.000000
상품분류ID,37.000000,58.000000,58.000000,56.000000,59.000000
첫 판매월,0.000000,0.000000,0.000000,0.000000,0.000000
대분류,5.000000,7.000000,7.000000,7.000000,7.000000


In [38]:
lag_features_to_clip

['월간 판매량_시차1', '월간 판매량_시차2', '월간 판매량_시차3']

**lag feature creation 2: 판매건수, 평균 판매가**

In [39]:
# create 3 month lag feature of 판매건수 by idx_features
all_data, lag_features_to_clip = add_lag_features(df=all_data, 
                                                  lag_features_to_clip=lag_features_to_clip,
                                                  idx_features=idx_features,
                                                  lag_feature='판매건수', 
                                                  nlags=3)

# create 3 month lag feature of 평균 판매가 by idx_features
all_data, lag_features_to_clip = add_lag_features(df=all_data, 
                                                  lag_features_to_clip=lag_features_to_clip,
                                                  idx_features=idx_features,
                                                  lag_feature='평균 판매가', 
                                                  nlags=3)

**lag feature creation 3: 평균 판매량**

In [40]:
# create lag feature of item_mean_features's each elem by idx_features
for item_mean_feature in item_mean_features:
    all_data, lag_features_to_clip = add_lag_features(df=all_data, 
                                                      lag_features_to_clip=lag_features_to_clip, 
                                                      idx_features=idx_features, 
                                                      lag_feature=item_mean_feature, 
                                                      nlags=3,
                                                      clip=True)

all_data = all_data.drop(item_mean_features, axis=1)

In [41]:
# create lag feature of shop_mean_feature's each elem by ['월ID', '상점ID', '상품분류ID']
for shop_mean_feature in shop_mean_features:
    all_data, lag_features_to_clip = add_lag_features(df=all_data,
                                                      lag_features_to_clip=lag_features_to_clip, 
                                                      idx_features=['월ID', '상점ID', '상품분류ID'], 
                                                      lag_feature=shop_mean_feature, 
                                                      nlags=3,
                                                      clip=True)

all_data = all_data.drop(shop_mean_features, axis=1)

**end of lag feature creation: deal with missing value**

In [42]:
all_data = all_data.drop(all_data[all_data['월ID'] < 3].index)

## Feature Engineering 6: Others

**add other features**

In [43]:
all_data['월간 판매량 시차평균'] = all_data[['월간 판매량_시차1',
                                          '월간 판매량_시차2', 
                                          '월간 판매량_시차3']].mean(axis=1)

In [44]:
all_data[lag_features_to_clip + ['월간 판매량', '월간 판매량 시차평균']] = all_data[lag_features_to_clip + ['월간 판매량', '월간 판매량 시차평균']].clip(0, 20)

In [45]:
all_data['시차변화량1'] = all_data['월간 판매량_시차1']/all_data['월간 판매량_시차2']
all_data['시차변화량1'] = all_data['시차변화량1'].replace([np.inf, -np.inf], 
                                                        np.nan).fillna(0)

all_data['시차변화량2'] = all_data['월간 판매량_시차2']/all_data['월간 판매량_시차3']
all_data['시차변화량2'] = all_data['시차변화량2'].replace([np.inf, -np.inf], 
                                                        np.nan).fillna(0)

In [46]:
all_data['신상여부'] = all_data['첫 판매월'] == all_data['월ID']

In [47]:
all_data['첫 판매 후 기간'] = all_data['월ID'] - all_data['첫 판매월']

In [48]:
all_data['월'] = all_data['월ID'] % 12

In [49]:
all_data = all_data.drop(['첫 판매월', '평균 판매가', '판매건수'], axis=1)

In [50]:
all_data = downcast(all_data, False) # data downcasting

## Feature Engineering 7: Final

In [51]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9904582 entries, 1122386 to 11026967
Data columns (total 31 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   월ID                      int8   
 1   상점ID                     int8   
 2   상품ID                     int16  
 3   월간 판매량                   int8   
 4   도시                       int8   
 5   상품분류ID                   int8   
 6   대분류                      int8   
 7   월간 판매량_시차1               int8   
 8   월간 판매량_시차2               int8   
 9   월간 판매량_시차3               int8   
 10  판매건수_시차1                 int8   
 11  판매건수_시차2                 int8   
 12  판매건수_시차3                 int8   
 13  평균 판매가_시차1               float64
 14  평균 판매가_시차2               float64
 15  평균 판매가_시차3               float64
 16  상품ID별 평균 판매량_시차1         float32
 17  상품ID별 평균 판매량_시차2         float32
 18  상품ID별 평균 판매량_시차3         float32
 19  상품ID 도시별 평균 판매량_시차1      float32
 20  상품ID 도시별 평균 판매량_시차2      float32
 21  상

In [52]:
# train data (feature)
X_train = all_data[all_data['월ID'] < 33]
X_train = X_train.drop(['월간 판매량'], axis=1)
# validation data (feature)
X_valid = all_data[all_data['월ID'] == 33]
X_valid = X_valid.drop(['월간 판매량'], axis=1)
# test data (feature)
X_test = all_data[all_data['월ID'] == 34]
X_test = X_test.drop(['월간 판매량'], axis=1)

# train data (target val)
y_train = all_data[all_data['월ID'] < 33]['월간 판매량']
# train data (target val)
y_valid = all_data[all_data['월ID'] == 33]['월간 판매량']

# garbage collection
del all_data
gc.collect();

## Train Model & Validation

In [53]:
import lightgbm as lgb

# LightGBM hyper parameter
params = {'metric': 'rmse', 
          'num_leaves': 255,
          'learning_rate': 0.005,
          'feature_fraction': 0.75,
          'bagging_fraction': 0.75,
          'bagging_freq': 5,
          'force_col_wise': True,
          'random_state': 10}

cat_features = ['상점ID', '도시', '상품분류ID', '대분류', '월']

# LightGBM trian & validation dataset
dtrain = lgb.Dataset(X_train, y_train)
dvalid = lgb.Dataset(X_valid, y_valid)
 
lgb_model = lgb.train(params=params,
                      train_set=dtrain,
                      num_boost_round=1500,
                      valid_sets=(dtrain, dvalid),
                      early_stopping_rounds=150,
                      categorical_feature=cat_features,
                      verbose_eval=100)        

[LightGBM] [Warning] Find whitespaces in feature_names, replace with underlines
[LightGBM] [Info] Total Bins 3886
[LightGBM] [Info] Number of data points in the train set: 9452298, number of used features: 30
[LightGBM] [Warning] Find whitespaces in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 0.297707
Training until validation scores don't improve for 150 rounds
[100]	training's rmse: 1.01082	valid_1's rmse: 0.987057
[200]	training's rmse: 0.909234	valid_1's rmse: 0.923085
[300]	training's rmse: 0.857871	valid_1's rmse: 0.898435
[400]	training's rmse: 0.829759	valid_1's rmse: 0.888834
[500]	training's rmse: 0.811326	valid_1's rmse: 0.885489
[600]	training's rmse: 0.797518	valid_1's rmse: 0.88463
[700]	training's rmse: 0.787457	valid_1's rmse: 0.885246
Early stopping, best iteration is:
[621]	training's rmse: 0.795255	valid_1's rmse: 0.884335


## Prediction & Submit Result

In [54]:
# prediction
preds = lgb_model.predict(X_test).clip(0, 20)

# create submission file
submission['item_cnt_month'] = preds
submission.to_csv('submission.csv', index=False)

In [55]:
del X_train, y_train, X_valid, y_valid, X_test, lgb_model, dtrain, dvalid
gc.collect();